<a href="https://colab.research.google.com/github/kdazero/NLP_emotion/blob/main/NLP_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install google.colab

In [10]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [11]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy


In [12]:
!nvidia-smi

Sat Feb  8 03:17:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              8W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [14]:
%cd /content/drive/My Drive/BDSE_NLP

/content/drive/My Drive/BDSE_NLP


In [15]:
!pip install torch torchvision torchaudio transformers datasets evaluate accelerate scikit-learn

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 

In [16]:
from datasets import load_dataset

ds = load_dataset("Johnson8187/Chinese_Multi-Emotion_Dialogue_Dataset")

emotion_mapping = {
    "平淡語氣": 0,
    "關切語調": 1,
    "開心語調": 2,
    "憤怒語調": 3,
    "悲傷語調": 4,
    "疑問語調": 5,
    "驚奇語調": 6,
    "厭惡語調": 7
}

# 應用映射到數據集
ds = ds.map(lambda x: {"emotion": emotion_mapping[x["emotion"]]})
print(ds["train"][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.18k [00:00<?, ?B/s]

data.csv:   0%|          | 0.00/315k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4159 [00:00<?, ? examples/s]

Map:   0%|          | 0/4159 [00:00<?, ? examples/s]

{'text': '你要不要去吃午餐？', 'emotion': 0}


In [17]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))

True
0


In [18]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

import random
from sklearn.metrics import f1_score


'''
函式
'''
# 讀取 .txt 文件
def load_dataset_from_file(file_path, seed=42):

    random.seed(seed)
    random.shuffle(file_path)

    sentences = []
    labels = []
    # 讀檔
    for text, index in zip(ds["train"]["text"], ds["train"]["emotion"]):
      sentences.append(text)
      labels.append(index)

    return sentences, labels

# 轉換成 huggingface trainer 可以使用的 datasets
def convert_to_dataset(sentences, labels, tokenizer, max_seq_length):
    # 建立 Dataset
    dataset = Dataset.from_dict({
        'sentences': sentences,
        'labels': labels
    })

    # 回傳切分資料 (訓練 和 驗證)
    dataset = dataset.train_test_split(test_size=0.2)
    '''
    print(dataset) 的內容如下:

    DatasetDict({
        train: Dataset({
            features: ['sentences', 'labels'],
            num_rows: 6212
        })
        test: Dataset({
            features: ['sentences', 'labels'],
            num_rows: 1554
        })
    })
    '''

    # 預處理資料
    def preprocess_data(dataset):
        # 將句子轉換為 token (tokenization)
        return tokenizer(
            dataset['sentences'],
            truncation=True,
            padding=True,
            return_tensors='pt',
            max_length=max_seq_length
        )

    # 轉換資料
    train_data = dataset['train'].map(preprocess_data, batched=True)
    valid_data = dataset['test'].map(preprocess_data, batched=True)

    return DatasetDict({
        'train': train_data,
        'test': valid_data
    })

# 計算模型評估指標
def compute_metrics(predicted_results):
    labels = predicted_results.label_ids
    preds = predicted_results.predictions.argmax(-1)
    '''
    為什麼是 argmax(-1)？

    import numpy as np
    predictions = np.array([
        [0.1, 0.9],
        [0.8, 0.2],
        [0.4, 0.6],
    ])
    preds = predictions.argmax(axis=-1)
    print(preds)

    np.argmax() 會找出最大的值，並回傳索引值
    - 如果是 axis=0，則會是 [1, 0]，因為由上而下，對每一個欄位取最大值
    - 如果是 axis=1，則會是 [1, 0, 1]，因為由左而右，對每一個列取最大值
    - 如果是 axis=-1，則會以最後一個維度為基礎，取最大值
    '''

    f1 = f1_score(labels, preds, average='weighted') # binary, micro, macro, weighted
    return {
        'f1': f1,
    }

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [19]:
from sklearn.metrics import f1_score

# 驗證 F1 score 算法
y_true = [0,0,1,1,1,0,0]
y_pred = [0,1,0,1,1,1,0]
# y_true = [0,2,1,2,1,0,1]
# y_pred = [0,1,0,2,1,1,2]
print(f1_score(y_true, y_pred, average='binary')) # binary, micro, macro

0.5714285714285714


In [20]:
if __name__ == "__main__":
    '''
    設定 hyperparameters
    '''
    model_name = 'google-bert/bert-base-chinese' # 預訓練模型名稱
    max_seq_length = 512 # 可訓練的序列最大長度
    num_labels = 8 # 二元分類
    output_dir = './output' # 輸出模型資料夾

    # 讀取訓練資料
    sentences, labels = load_dataset_from_file(ds)

    # 載入 tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 將資料轉換為 huggingface 可以使用的格式
    dataset = convert_to_dataset(
        sentences,
        labels,
        tokenizer,
        max_seq_length
    )

    # 讀取模型
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels
    )

    # 設定訓練參數
    training_args = TrainingArguments(
        output_dir=output_dir, # 輸出資料夾
        overwrite_output_dir=True,
        num_train_epochs=3, # 訓練回合數
        per_device_train_batch_size=16, # 批次大小
        per_device_eval_batch_size=16, # 批次大小
        gradient_accumulation_steps=2,
        learning_rate=0.00003, # 學習率 5e-5
        warmup_steps=100,
        weight_decay=0.01,
        eval_strategy="steps", # epoch, steps, no
        eval_steps=10,
        save_strategy="steps", # epoch, steps, no
        save_steps=10,
        save_total_limit=2,
        load_best_model_at_end=True,
        seed=42, # 隨機種子
        # lr_scheduler_type="linear", # https://blog.csdn.net/muyao987/article/details/139319466
        # report_to='wandb', # https://wandb.ai/
    )

    # 設定 Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        data_collator=DataCollatorWithPadding(tokenizer),
        compute_metrics=compute_metrics,
    )

    # 開始訓練
    trainer.train()

    # 儲存模型
    trainer.save_model(output_dir) # , safe_serialization=True

    # 儲存 tokenizer
    tokenizer.save_pretrained(output_dir)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

Map:   0%|          | 0/3327 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bob0109-mail (bob0109-mail-ispan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss,F1
10,No log,2.086766,0.048507
20,No log,1.992221,0.163533
30,No log,1.872863,0.261981
40,No log,1.707362,0.330645
50,No log,1.509823,0.459573
60,No log,1.307756,0.626042
70,No log,1.089637,0.727673
80,No log,0.883867,0.772189
90,No log,0.793648,0.773598
100,No log,0.678625,0.810431


In [39]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    pipeline,
)
from pprint import pprint

model_dir = './output'
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
pipe = pipeline(task='text-classification', model=model, tokenizer=tokenizer, device=0)

list_text = [
  "我每天都能跟她一起上學，我好開心！",
	"最好的朋友要離開臺灣了，以後可能不容易再見面...",
	"我覺得我快不行了...",
	"剛剛收到研究所錄取的通知書！",
	"今年的冬天好像比較晚來。"
]
result = pipe(list_text)
reverse_emotion_mapping = {v: k for k, v in emotion_mapping.items()}

for i in range(5):
  print("="*50)
  print(f"文字:{list_text[i]}")
  label_number = int(result[i]['label'][-1])
  print(f"心情:{reverse_emotion_mapping[label_number]}")
  print(f"分數:{result[i]['score']}")



Device set to use cuda:0


文字:我每天都能跟她一起上學，我好開心！
心情:開心語調
分數:0.9884305000305176
文字:最好的朋友要離開臺灣了，以後可能不容易再見面...
心情:悲傷語調
分數:0.9803542494773865
文字:我覺得我快不行了...
心情:悲傷語調
分數:0.9617601633071899
文字:剛剛收到研究所錄取的通知書！
心情:開心語調
分數:0.9759320020675659
文字:今年的冬天好像比較晚來。
心情:平淡語氣
分數:0.9782320857048035
